### converting pixeldata to png (수정 필요)

0) 3D 엑스레이 장비 데이터용

1) 작업 폴더 복사

2) 복사한 폴더에 image 폴더 생성

3) image 폴더에 이미지 파일 저장

4) 이미지 변환 완료 후 binary 폴더 삭제

5) 파일 이름 변경

In [10]:
import shutil
import cv2
import os
import numpy as np
import pandas as pd
import natsort

In [ ]:
df = pd.read_excel('C:\\Users\\user\\Desktop\\국토부과제_2차년도_엑스레이 이미지 데이터 DB_ver0.1.xlsx')

dir_working = 'C:\\Users\\user\\Desktop\\raw data\\round 11' # 촬영 데이터가 모여져 있는 폴더 경로

i = 4338

dir_working_copy = dir_working + '\\이미지 변환'

if not os.path.exists(dir_working_copy):
    shutil.copytree(dir_working, dir_working_copy) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.

filelist = os.listdir(dir_working_copy)

for file in filelist:
    os.chdir(dir_working_copy)
    # read image as binary
    read_binary = open(dir_working_copy+'/'+file,"rb")
    img = np.fromfile(read_binary, dtype = 'uint16', sep = "", count = (896*760), offset = 0) # dtype, count는 상황에 맞게 수정 필요
    read_binary.close()

    img = np.reshape(img, [760,896]) # 810 -> height, 896 -> width

    markfile = df.values[i][-1] + '.png'
    cv2.imwrite(markfile, img)
    os.chdir(dir_working_copy)

    print(file)
    print(markfile)

    i = i + 1

    if os.path.exists(dir_working_copy + '/' + file):
        os.remove(dir_working_copy +'/'+ file) # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제함.
        


In [35]:
# 실사 이미지 이름 변경

df = pd.read_excel('C:\\Users\\user\\Desktop\\국토부과제_2차년도_엑스레이 이미지 데이터 DB_ver0.1.xlsx')

dir_working = 'C:\\Users\\user\\Desktop\\실사이미지\\Round 11'

filelist = os.listdir(dir_working)

filelist = natsort.natsorted(filelist)

for file in filelist:
    newname = df.values[9 * (int(file.split(".")[0]) - 1)][-1][:-2]
    os.rename(dir_working + "/" + file, dir_working + "/" + newname + ".jpg")